In [ ]:
!pip install hausastemmer

In [ ]:
import pandas as pd
import json
import re
import hausastemmer

# Загрузка словарей

In [ ]:
with open('pron.json', encoding='utf-8') as f:
    dictionary_pronouns = json.load(f)

with open('document_lemm.json', encoding='utf-8') as f:
    dictionary_full = json.load(f)

#with open('verbs.json', encoding='utf-8') as f:
#with open('nouns.json', encoding='utf-8') as f:
#with open('other.json', encoding='utf-8') as f:

#  Лемматизатор

## Обработка числительных

In [ ]:
def process_numbers(sentence):
    numerals = {
        'ɗaya': 1, 'biyu': 2, 'uku': 3, 'huɗu': 4, 'biyar': 5,
        'shida': 6, 'bakwai': 7, 'takwas': 8, 'tara': 9,
        'goma': 10, 'ashirin': 20, 'talatin': 30, 'arba′in': 40,
        'hamsin': 50, 'sittin': 60, 'saba′in': 70, 'tamanin': 80,
        'casa′in': 90, 'ɗari': 100, 'metan': 200, 'alif': 1000,
        'dubu': 1000
        }

    # заменяем все числа на 0
    sentence = ['0' if word in numerals else word for word in sentence]

    if sentence.count('0') > 1:
        line = ' '.join(sentence)

        # удаляем sha в последовательностях "0 sha 0"
        line = re.sub(r'0\s+sha\s+0', '0', line)

        # удаляем da в последовательностях "0 da 0 (... da 0)"
        line = re.sub(r"\b0(?:\s+da\s+0)+\b", "0", line)

        sentence = line.split()

    # помечаем измененные сущ. (кроме измерений) множественным числом
    measures = ['kilomita', 'tan', 'lita', 'miniti', 'mako']

    for i in range(1, len(sentence)):
        if sentence[i] == 0 and sentence[i-1] == 'na':
            sentence[i-1] == '_gen'
        elif sentence[i] == 0 and (sentence[i-1] not in measures):
            sentence[i-1] += '/pl'

    return sentence

## Обработка самостоятельных местоимений

In [ ]:
# для односоставных мест.
def process_pron(word1, pronouns=dictionary_pronouns):

    for pron_category, pron_dict in pronouns.items():
        if word1 in pron_dict:
            value = pron_dict[word1]

            # если глагольное местоимение, возвращаем verbal_pron
            if pron_category.startswith('verbal'):
                word1 = f'verbal_pron_{value}'

            elif pron_category.startswith('neg'):
                word1 = f'ba+verbal_pron_{value}'

            # во всех остальных случаях возвращаем pron
            else:
                word1 = f'pron_{value}'

    return word1


# для двусоставных мест.
def process_pron2(word1, word2, pronouns=dictionary_pronouns):

    # склеиваем два слова в одно
    word_whole = word1 + ' ' + word2

    # ищем в словарях
    for pron_category, pron_dict in pronouns.items():
        if word_whole in pron_dict:
            value = pron_dict[word_whole]
            word_whole = f'pron_{value}'
            return word_whole, '0'

    return word1, word2

## Обработка глаголов

In [ ]:
def process_verb(word, verbs=dictionary_full):
    changes_vow = {'i': 'a', 'e': 'a'}
    changes_cons = {'c': 't', 'j': 'z', 'sh': 's'}

    last_char = word[-1]
    new_word = word

    # конструируем инфинитив для возможных транзитивных глаголов
    if last_char in changes_vow:
        new_word = word[:-1] + changes_vow[last_char]

        prev_char = word[-2]
        if prev_char in changes_cons:
            new_word = new_word[:-2] + changes_cons[prev_char] + new_word[-1:]
        elif len(word) >= 3 and word[-3:-1] == 'sh':
            new_word = new_word[:-3] + 's' + new_word[-2:]

    # если нашли измененный, возвращаем его
    if new_word in verbs:
        if 'tr' in verbs[new_word]:
            return new_word + '_tr'
        else:
            return new_word + '_poss_tr'

    # если нашли неизмененный, возвращаем его
    if word in verbs:
        if 'intr' in verbs[word]:
            return word + '_intr'
        else:
            return word + '_poss_intr'

    return word

## Обработка генитивных конструкций (с сущ. и мест.)

In [ ]:
# обработка генитивных конструкций

# cущ. + сущ.
def process_gen(word, marked_gen=False, nouns=dictionary_full):
    possible_outcomes = []
    possible_outcomes1 = []
    keeping = False

    if word[-3:-1] == '/pl':
      keeping = True
      word = word[:-3]

    # если есть генитивная связка
    if word == ('na' or 'ta'):
        return '_gen'

    # если есть суффикс
    if word[-2:] in ['na', 'ra']:
        possible_outcomes = [word[-2:], word[-2:] + 'i', word[-2:] + 'u']

    # обработка результата

    # если не указано, что мн. ч.
    if not keeping:
        for item in possible_outcomes:
            if item in nouns:
                possible_outcomes1.append(item)

        if possible_outcomes1:
            return possible_outcomes1[0]

    # если указано, что мн. ч.
    if keeping:
        word += '/pl'

    return word

# сущ. + посесс. мест.
def process_gen2(word, nouns=dictionary_nouns):

    pron_affix = {
        'na': '1sg/m', 'nmu': '1pl/m',
        'nka': '2m/m', 'nki': '2f/m', 'nku': '2pl/m',
        'nsa': '3m/m', 'nta': '3f/m', 'nsu': '3pl/m',
        'ta': '1sg/f', 'rmu': '1pl/f',
        'rka': '2m/f', 'rki': '2f/f', 'rku': '2pl/f',
        'rsa': '3m/f', 'rta': '3f/f', 'rsu': '3pl/f'
        }

    for form in pron_affix:
        if word.endswith(form):
            new_word = word[:len(form)]
            return new_word

    return word

## Обработка плюралиса

In [ ]:
# функция для нахождения числа слогов в слове
def count_syllables(word):

    vow = ['au', 'ai', '’ya', 'a', 'e', 'i', 'y', 'o', 'u']
    pattern = '|'.join(map(re.escape, vow))
    matches = re.findall(pattern, word, flags=re.IGNORECASE)

    return len(matches)

In [ ]:
def process_plural(word, nouns=dictionary_nouns):
    possible_outcomes = []

    syllables = count_syllables(word)
    # проверяем, может ли быть суффикс

    # суфф. a (двухсложные слова на -o/-e м.р. кроме mace (ж.р.) -> mata)
    if word[-1] == 'a':
        if syllables == 2:
            new_word = word[-1]
            possible_outcomes += [new_word + 'e', new_word + 'o']

    # суфф. i (двух- и трехсложные слова)
    if word[-1] == 'i':
        if 2 <= syllables <= 3:
        possible_outcomes.append(word[-1])

    # суфф. u
    if word[-1] == 'u':
        possible_outcomes.append(word[-1])

    # суфф. ai (многосложные слова,
    # последний согласный слова может депалатализовываться)
    if word[-2:] == 'ai':
        if syllables >= 3:
            possible_outcomes.append(word[:-2])

    # суфф. aye, aki, una, uwa
    if word[-3:] in ['aye', 'aki', 'una', 'uwa']:
        possible_outcomes.append(word[:-3])

    # суфф. anni
    if word[-4:] == 'anni':
        possible_outcomes.append(word[:-4])

    # проверяем, есть ли в noun или other слово, которое
    # начинается с последовательнстей из possible_outcomes

    for noun_category, noun_dict in nouns.items():
        if word in noun_dict:
            return word
    return word + '_not_found'

## Обработка предложения

In [ ]:
def process_sent(sentence):

    # удаляем числа (заменяем на 00), разбиваем предложение на слова, удаляем пунктуацию
    sentence = re.sub(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", "00", sentence)
    sentence = [word.lower() for word in re.sub(r"\B['-]|['-]\B|[^\w\s'’-]", '', sentence).split()]

    # очищаем предложение от числительных (заменяем на 0)
    sentence = process_numbers(sentence)

    # заменяем все самостоятельные местоимения
    for word0 in range(len(sentence)-1):
      sentence[word0], sentence[word0+1] = process_pron2(sentence[word0], sentence[word0+1])

    sentence = [process_pron(i) for i in sentence]

    # приводим все глаголы к начальному виду
    next_verb = False
    for word1 in range(len(sentence)):
        if ('verbal' in sentence[word1]):
            next_verb = True
        elif sentence[word1] == 'kuma':
            continue  # между приглаг. мест. и глаг. может быть только kuma
        else:
            if next_verb:
                sentence[word1] = process_verb(sentence[word1], dictionary_verbs)
                next_verb = False

    # обрабатываем генитив
    word2_processed = ''
    for word2 in range(len(sentence)-1):
        if not '_' in sentence[word2]:
            word2_processed = process_gen2(sentence[word2])
        if word2_processed == sentence[word2]:
            if not '_' in sentence[word2+1]:
                if sentence[word2+1] == ('nan' or 'can'):
                    sentence[word2] = process_gen(True, sentence[word2])
                else:
                    sentence[word2] = process_gen(sentence[word2])

    # обрабатываем плюралис
    for word3 in range(len(sentence)-1):
        if not '_' in sentence[word3]:
          sentence[word3] = process_plural(sentence[word3])

    # удаляем все _
    # sentence = [word.split('_')[0] if not word.startswith('verbal') else 'pron' for word in sentence]

    return sentence


#   Процессинг текстов

In [ ]:
s = '-	Aure! Inna, ni fa na gaya muku, ba zan auri kowa ba, sai wanda nake so. Kun san, zamani ya sake. Kuma yanzu ban ga abin da zai hana ku ba ni shi ba. Yana da mutumci da natsuwa. Kuma daidai muke, tun da yana da asali, ba za ku yarda in zaɓi na ƙasa da ni ba? - Bodado, mu ba mu hana ki zaɗenki ba. Ba mu kuma ce ba shi da duk abin da kika ce yana da shi ba. Mu dai ba irinmu ba ne. Hanyoyinsu da ɗabi’o’insu da al’adunsu sun bambanta da namu. Yaya za mu yi mu ba shi ɗiyarmu? Ba ma haka ba, idan wata rana ya ce za shi garinsu, yaya za mu yi mu hana shi tafiya da matarsa?'
s1 = "A farko-farkon ƙarni na 19, a sa'adda ƙabilar Hausawa ke yunƙurin kawar da mulkin aringizo na Fulani, sai Turawan Mulkin Mallaka na Birtaniya suka mamaye Arewancin Najeriya, da kuma kafa manufofin mulkinsu, a bisa karkashin mulkin Birtaniya, 'yan mulkin mallaka sai suka marawa Fulani baya na cigaba da manufofin aringizon siyasarsu, har yanzu dai mulkin gamin-gambiza tsakanin Hausawa da Fulani shi ne ya yi kane-kane a arewacin Najeriya."

s = s1.split('.')
for i in s:
    print(i)
    print(*process_sent(i))

print(process_sent(s3))

# Векторизация

to be continued...